# Tutorial 06 - Working with Electrical Stimulation Data

In addition to pyCAP.Ephys for loading in electrophysiology data, pyCAP also has a class, pyCAP.Stim, for working with stimulation data from different vendors in a consistent format. This tutorial covers the basics of loading in and working with stimulation data that is paired with an electrophysiology data set.

First, we will load in an electrophysiology data set.

In [9]:
# import statements
import pyCAP
import matplotlib.pyplot as plt

# put matplotlib in widget mode so that we can use interactive plots
%matplotlib widget 

# path to directory containing the TDT tank
directory = 'C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/pnpig200113-200113-142737'
data = pyCAP.Ephys(directory)

# Starting by removing the excess channel 'RawG 4'. We could do this either by the index in the channel list (7) or using the channel name directly. 
data = data.remove_ch('RawG 4')

# We can rename the channels using the set_ch_names method and providing a list of new channel names that matches the number of channels in the data set.
data = data.set_ch_names(['LIFE 1', 'LIFE 2', 'LIFE 3', 'LIFE 4', 'EMG 1', 'EMG 2', 'EMG 3'])

# Lets also set the types of the individual channels so that we can indepentently interact with LIFE and EMG channels
data = data.set_ch_types(['LIFE']*4 + ['EMG']*3)

read from t=0s to t=855.82s


We can load in a stimulation data set in two ways. The first is by passing a file path for the data set contining stimulation infomation. For TDT data sets, stimulation data is stored in the same tdt tank as electrophysiology data and we can pass the same directory path to pyCAP.Stim as we passed to pyCAP.Ephys.

In [10]:
stim_data = pyCAP.Stim(directory)

read from t=0s to t=855.82s


***TODO: Right now loading stim data directly from any other previously loaded pycap abject does not work as expected. For now data must be loaded from a file path or list of files.***

The second way that we could load in a stimulation data set that corresponds to an already loaded pyCAP object, such as a pyCAP.Ephys object, is to pass the pyCAP object to the pyCAP.Stim constructor. In this case, *data* is a pyCAP.Ephys object and we could have initialized pyCAP.Stim with the following code.

        stim_data = pyCAP.Stim(data)

## Exploration of Stimulation Data

When you use pyCAP to read in electrical stimulation data, it puts data from different input data types into a consistent format that is easy to interact with.

Let's start by looking at the the paramters that were applied during this experiment.

In [11]:
stim_data.parameters

onset time (s)  offset time (s)  period (ms)  frequency (Hz)  \
0 0        6.116844        36.130282    40.017918       24.988806   
  1       96.116859       126.130297    40.017918       24.988806   
  2      186.116874       216.130312    40.017918       24.988806   
  3      276.116890       306.130328    40.017918       24.988806   
  4      366.116905       396.130343    40.017918       24.988806   
  5      456.116920       486.130359    40.017918       24.988806   
  6      546.116936       576.130374    40.017918       24.988806   
  7      636.116951       666.130389    40.017918       24.988806   
  8      726.116966       756.130405    40.017918       24.988806   
  9      816.116982       846.130420    40.017918       24.988806   

     pulse count  pulse amplitude (μA)  duration (ms)  pulse duration (ms)  \
0 0          750                 -50.0   30013.438225                  0.4   
  1          750                -400.0   30013.438225                  0.4   
  2          750               -1000.0   30013.438225                  0.4   
  3          750                -300.0   30013.438225                  0.4   
  4          750                -200.0   30013.438225                  0.4   
  5          750                -500.0   30013.438225                  0.4   
  6          750                -150.0   30013.438225                  0.4   
  7          750                -100.0   30013.438225                  0.4   
  8          750               -2000.0   30013.438225                  0.4   
  9          750               -3000.0   30013.438225                  0.4   

     interphase delay (ms)  channel  
0 0                    0.0        1  
  1                    0.0        1  
  2                    0.0        1  
  3                    0.0        1  
  4                    0.0        1  
  5                    0.0        1  
  6                    0.0        1  
  7                    0.0        1  
  8                    0.0        1  
  9                    0.0        1

This returns a human readable table of stimulation paramters. For people familiar with the standard scientific computing libraries in python, stim_data.parameters return a pandas DataFrame. Useful infromation on working with pandas DataFrames can be found here: https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#min-tut-03-subset. 

In pyCAP we generally work with stimulation data by referencing the index of a particular stimulation within the stim_data.parameters table. Using the indexing methods available through pandas makes it easy to obtain and work with a subset of stimulation parameters, work with parameters sorted on a particular value(s), etc. You will see this in more detail within both this tutorial and Tutorial 7 on Evoked Compound Action Potential analysis. 

We can easily plot our stimulation data alongside the relevent electrophysiology data using the same plot method we have seen in previous tutorials. This is a nice way to interactively explore data and view the effects of stimulation. To do this, we simply have to pass stim_data to the events attribute of the plot method.

In [12]:
data.plot(x_lim=(0, 2), events=stim_data, show='notebook')

AppLayout(children=(Output(layout=Layout(grid_area='header')), FloatSlider(value=0.0, description='Start Time:…

Now you can see stimulation events on the scrollbar axis of the plot (grey bars) as well as a new event axis at the top of the plot showing the both the stimuation events (grey pulses) but also the individual event data for stimulation pulses (orange line). Scroll through the data set to see both of these.

## Working with Multiple Stimulation Data Sets

Similar to electrophysiology data multiple stimulation data sets can be read in and worked with simultaneously. To show this, let's use our same code from Tutorial 03 on working with multiple data sets to read in a list of TDT data sets. 

In [13]:
from glob import glob # Import glob for file handling
from pprint import pprint # For pretty printing

# Empty list we will append tdt tanks to
tdt_tanks = []

# Create list of directories contining TDT data
data_folder = "C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/20191204/Data/Imthera_Pig_Exeriment_25Hz-191204"
directory_search = "/*/"
directories = glob(data_folder +  directory_search)

# TDT tanks are a directory with a number of files. In particular any stream data is contained in a *.tev file.
# Let's use that as in indicator of a complete tdt tank.
tev_search = "/*.tev"
for d in directories:
    tev_files = glob(d + tev_search)
    # Append 
    if len(tev_files) > 0:
        tdt_tanks.append(d)

# Print the list of file paths we will be loading
pprint(tdt_tanks)

['C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/20191204/Data/Imthera_Pig_Exeriment_25Hz-191204\\pnpig191126-191204-114445\\',
 'C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/20191204/Data/Imthera_Pig_Exeriment_25Hz-191204\\pnpig191126-191204-120053\\',
 'C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/20191204/Data/Imthera_Pig_Exeriment_25Hz-191204\\pnpig191126-191204-121938\\',
 'C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/20191204/Data/Imthera_Pig_Exeriment_25Hz-191204\\pnpig191126-191204-123528\\',
 'C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/20191204/Data/Imthera_Pig_Exeriment_25Hz-191204\\pnpig191126-191204-125241\\',
 'C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/20191204/Data/Imthera_Pig_Exeriment_25Hz-191204\\pnpig191126-191204-131039\\',
 'C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/20191204/Data/Imthera_Pig_Exeriment_25Hz-191204\\pnpig191126-191204-132809\\',
 'C:/Users/james/OneDrive/Documents/Data/pyCAP/TDT/20191204/Data/Imthera_Pig_Exeriment_25H

Now we can easily load in both the electrophysiology data and the stimulation data from this list of directories.

In [14]:
m_data = pyCAP.Ephys(tdt_tanks)
m_stim_data = pyCAP.Stim(tdt_tanks)

read from t=0s to t=863.46s
read from t=0s to t=886.78s
read from t=0s to t=912.95s
read from t=0s to t=869.83s
read from t=0s to t=855.82s
read from t=0s to t=860.44s
read from t=0s to t=872.68s
read from t=0s to t=893.99s
read from t=0s to t=849.06s
read from t=0s to t=849.87s
read from t=0s to t=863.46s
read from t=0s to t=886.78s
read from t=0s to t=912.95s
read from t=0s to t=869.83s
read from t=0s to t=855.82s
read from t=0s to t=860.44s
read from t=0s to t=872.68s
read from t=0s to t=893.99s
read from t=0s to t=849.06s
read from t=0s to t=849.87s


If we take a look at the parameter list for *m_stim_data*, we now have 10 groups of stimulation paramters corresponding to the 10 data sets we loaded into the pyCAP.Ephys and pyCAP.Stim data sets.

In [17]:
m_stim_data.parameters

onset time (s)  offset time (s)  period (ms)  frequency (Hz)  \
0 0        7.811973        37.825411    40.017918       24.988806   
  1       97.811988       127.825427    40.017918       24.988806   
  2      187.812004       217.825442    40.017918       24.988806   
  3      277.812019       307.825457    40.017918       24.988806   
  4      367.812035       397.825473    40.017918       24.988806   
..              ...              ...          ...             ...   
9 5      454.846054       484.859493    40.017918       24.988806   
  6      544.846070       574.859508    40.017918       24.988806   
  7      634.846085       664.859523    40.017918       24.988806   
  8      724.846100       754.859539    40.017918       24.988806   
  9      814.846116       844.859554    40.017918       24.988806   

     pulse count  pulse amplitude (μA)  duration (ms)  pulse duration (ms)  \
0 0          750               -2000.0   30013.438225                  0.4   
  1          750                -300.0   30013.438225                  0.4   
  2          750                -200.0   30013.438225                  0.4   
  3          750                -500.0   30013.438225                  0.4   
  4          750                -150.0   30013.438225                  0.4   
..           ...                   ...            ...                  ...   
9 5          750               -3000.0   30013.438225                  0.4   
  6          750               -1000.0   30013.438225                  0.4   
  7          750                 -50.0   30013.438225                  0.4   
  8          750                -100.0   30013.438225                  0.4   
  9          750                -400.0   30013.438225                  0.4   

     interphase delay (ms)  channel  
0 0                    0.0        1  
  1                    0.0        1  
  2                    0.0        1  
  3                    0.0        1  
  4                    0.0        1  
..                     ...      ...  
9 5                    0.0        1  
  6                    0.0        1  
  7                    0.0        1  
  8                    0.0        1  
  9                    0.0        1  

[100 rows x 10 columns]